# SimpleConf Fixture Walkthrough
Use the bundled fixtures to explore how layering and ordering change the final configuration.


In [ ]:
from pathlib import Path
import os
from pprint import pprint

from simpleconf import load

FIXTURES = Path('tests/fixtures').resolve()
BASE = FIXTURES / 'base'
LOCAL = FIXTURES / 'local'
PROD = FIXTURES / 'prod'

print(f'Base fixtures: {BASE}')
print(f'Local fixtures: {LOCAL}')
print(f'Prod fixtures: {PROD}')


## Load only base
Base defaults include transport fallbacks, channel flags, and limits.


In [ ]:
base_only = load(layers=[BASE])
pprint(base_only.messaging.to_dict())


## Load base and local
Local overrides introduce new keys (e.g. `channels.push`) and override existing ones.


In [ ]:
os.environ['MESSAGING_WEBHOOK'] = 'https://hooks.notebook/local'
base_local = load(layers=[BASE, LOCAL])
pprint(base_local.messaging.to_dict())


## Load base, local, and prod
The prod layer adds alert metadata and overrides transport/limit settings (including JSON data).


In [ ]:
os.environ.pop('MESSAGING_WEBHOOK', None)
base_local_prod = load(layers=[BASE, LOCAL, PROD])
pprint({
    'transport': base_local_prod.messaging.transport.to_dict(),
    'channels': base_local_prod.messaging.channels.to_dict(),
    'alerts': base_local_prod.messaging.alerts.to_dict(),
    'limits': base_local_prod.messaging.limits.to_dict(),
})


## Change layer order
Later layers win. Swapping the order restores base defaults and drops prod-only keys.


In [ ]:
ordered = load(layers=[BASE, LOCAL, PROD])
reordered = load(layers=[PROD, LOCAL, BASE])
print('Ordered transport.primary:', ordered.messaging.transport.primary)
print('Reordered transport.primary:', reordered.messaging.transport.primary)
print('Ordered transport.backup:', ordered.messaging.transport.backup)
print('Reordered transport.backup:', reordered.messaging.transport.backup)
print('Voice channel present (ordered):', ordered.messaging.channels.get('voice'))
print('Voice channel present (reordered):', reordered.messaging.channels.get('voice'))


## Inspect prod JSON contribution
The JSON file injects the burst limit and pager-style backup targets.


In [ ]:
print('Burst limit:', base_local_prod.messaging.limits.burst)
print('Backup channel:', base_local_prod.messaging.transport.backup)
print('Pager alert enabled:', base_local_prod.messaging.alerts.pager)
